# Chapter 9

### 使用主成份縮減特徵(且保留了變異)

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

digits = datasets.load_digits()
# 特徵矩陣標準化
features = StandardScaler().fit_transform(digits.data)
# 產生能保留99%變異的PCA
pca = PCA(n_components=.99,
                whiten=True) # 會對每一個主成分的值進行轉換，如此就能有0平均(zero-mean)與單位變異(unit variance)
                # svd_solver = "randomized" # 實做了一個尋找第一主成分的隨機演算法(stochastic algorithm)為節省時間
# activate
pca_features = pca.fit_transform(features)

print("Original number of features :", features.shape[1])
print("Reduced number of features :", pca_features.shape[1])

Original number of features : 64
Reduced number of features : 54


#### Notes: PCA是一種非監督的技術，即不需要目標向量的資訊，只需考慮特徵矩陣即可。另外，PCA所產生的新特徵無法由人類來解讀，若要保留解釋模型的能力，以特徵選取來進行降維會比較好。
![線性分離](img/78666551_595455054560185_8155747298551267328_n.jpg)

### 為不可線性分離之資料作特徵縮減

In [10]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles
# 產生線性不可分離資料
features, _ = make_circles(n_samples=1000, random_state=1, noise=0.1, factor=0.1)
# 套用帶徑向基函式(radius basis function, RBF)
kpca = KernelPCA(kernel="rbf", gamma=15, n_components=1)
features_kpca = kpca.fit_transform(features)

print("Original number of features :", features.shape[1])
print("Reduced number of features :", features_kpca.shape[1])

Original number of features : 2
Reduced number of features : 1


#### Notes: 標準的PCA使用線性投影以縮減特徵，若資料為線性可分，則PCA可以運作得很好，若為線性不可分則單用PCA效果較差，使用投影降維法的時候會使分類交織在一起，但是我們希望縮減為度也可以使資料線性平分，PCA_Kernel是我們的好幫手(rbf, poly, sigmoid...)，但是PCA_Kernel必須定義參數數量(如n_components=1)，還要設定Kernel本身的參數。
![不可線性分離](img/77410285_2148243928817543_140205286072778752_n.jpg)

### 運用類型可分性最大化縮減特徵(By LDA)

In [11]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
iris = datasets.load_iris()
features = iris.data
target = iris.target
# 產生並執行LDA，然後用它來轉換特徵
lda = LinearDiscriminantAnalysis(n_components=1) # 代表要傳回的特徵數
features_lda = lda.fit(features, target).transform(features)

print("Original number of features :", features.shape[1])
print("Reduced number of features :", features_lda.shape[1])

Original number of features : 4
Reduced number of features : 1


In [12]:
# 檢視每一成份所解釋的變異數量
lda.explained_variance_ratio_

array([0.9912126])

#### Notes: LDA是一種分類方法也是一種常用的維度縮減技術，相較於PCA，PCA我們只對能將資料中變異最大化的成分感興趣，而在LDA中我們還有將分類間的差異最大化這個額外的目標。
![LDA](img/78144320_595190831217947_8467876655149350912_n.jpg)

#### Notes: n_components的調參值技巧，先將它設為None，在用LDA傳回每一成分所解釋的變異比，是否大於threshold。

In [17]:
lda = LinearDiscriminantAnalysis(n_components=None)
features_lda = lda.fit(features, target)

lda_var_ratios = lda.explained_variance_ratio_
def select_n_components(var_ratio, goal_var: float) -> int:
    total_variance = 0.0
    # init
    n_components = 0
    for explained_variance in var_ratio:
        total_variance += explained_variance
        n_components += 1
        if total_variance >= goal_var:
            break
    return n_components
select_n_components(lda_var_ratios, 0.95)

1

### 運用矩陣分解縮減特徵-非負值特徵矩陣(By NMF)

In [19]:
from sklearn.decomposition import NMF
from sklearn import datasets

digits = datasets.load_digits()
features = digits.data

nmf = NMF(n_components=10, random_state=1)
feature_nmf = nmf.fit_transform(features)
print("Original number of features :", features.shape[1])
print("Reduced number of features :", feature_nmf.shape[1])

Original number of features : 64
Reduced number of features : 10


#### Notes: NMF是線性維度縮減的非監督式學習，其能將特徵矩陣分解(即拆解成幾個相乘後與原矩陣相近的矩陣)成代表觀察與其特徵之潛在關係的矩陣。(拆解過後得矩陣維度會明顯小於相乘後的)給定回傳所需的特徵數，r
$$ V \sim WH $$
$$V是d \times n的特徵矩陣(d個特徵,n個觀察,且不能有負值)$$
$$W是d \times r的矩陣$$
$$H是r \times n的矩陣$$
$$透過r值的調整，我們可以設定需要縮減的維度$$

#### NMA特徵矩陣中不能有負值，也不會給我們輸出特徵的explained variance)

### 在稀疏資料上縮減特徵

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
import numpy as np

In [6]:
digits = datasets.load_digits()
# 特徵標準化
features = StandardScaler().fit_transform(digits.data)
# 產生稀疏矩陣
features_sparse = csr_matrix(features)
# 產生TSVD
tsvd = TruncatedSVD(n_components=10)
# fit_transform
features_sparse_tsvd = tsvd.fit(features_sparse).transform(features_sparse)
# results
print("Original number of features :", features_sparse.shape[1])
print("Reduced number of features :", features_sparse_tsvd.shape[1])

Original number of features : 64
Reduced number of features : 10


#### Notes: TSVD專門處理稀疏矩陣，PCA其實常在一個步驟中運用非截斷奇異值(SVD)。在正規的SVD中，給定d個特徵，SVD會產生d * d的因子矩陣，而TSVD則傳回n * n個(透過參數設定)

In [9]:
tsvd.explained_variance_ratio_[:3].sum() # 前三個輸出成分解釋了約30%的原始資料變異

0.3003938535247808

In [14]:
# 產生並執行帶有比特徵少1的TSVD
tsvd = TruncatedSVD(n_components=features_sparse.shape[1]-1)
features_tsvd = tsvd.fit(features)

tsvd_var_ratios = tsvd.explained_variance_ratio_
def select_n_components(var_ratio, goal_var):
    # 設定初始已釋變異
    total_variance = 0.0
    # 設定初始特徵數
    n_components = 0
    for explained_variance in var_ratio:
        total_variance += explained_variance
        n_components += 1
        if total_variance >= goal_var:
            break
    return n_components
select_n_components(tsvd_var_ratios, 0.95)

40

# Chapter 10

### Feature selection : Filter, Wrapper, Embedded
* Filter: 透過檢視統計性質來選取最好的特徵
* Wrapper: 透過試誤法，找到能產生最高預測品質之模型的特徵子集
* Embedded: 透過選取最佳特徵子集作為學習演算法訓練過程

### 設定特徵變異門檻(移除低變異)